In [1]:
import pandas as pd
import psycopg2
import os
import json
import datetime

In [2]:
with open("/home/anas/notebook/UseCase/config.json") as f:
    config = json.load(f)
print("Config:",config)

Config: {'database': 'datamart', 'user': 'postgres', 'password': 'intelrebel', 'host': 'localhost', 'port': '5432'}


In [8]:
#read config file in input
with open("/home/anas/notebook/UseCase/input_config.txt") as f:
    input_file=f.readline()
print("will process:",input_file)

will process: /home/anas/notebook/UseCase/input/meetingAttendanceReport(General).csv


In [9]:
meta_data=[]
log_call=[]
lines=[]
with open(input_file,encoding="utf-16") as f:
    lines=f.readlines()
meta_data=lines[0:5]
log_call=lines[6:]

In [32]:
course = "DWH"
course

'DWH'

In [33]:
with open(f"/home/anas/notebook/UseCase/output/log_{input_file.split('/')[-1]}","w") as f:
    for value in log_call:
        f.write(value)
    f.close()
df_log = pd.read_csv(f"/home/anas/notebook/UseCase/output/log_{input_file.split('/')[-1]}",sep="\t")
df_log

,Nom complet,Heure d'arrivée,Heure de départ,Durée,E-mail,Rôle
0,Imade BENELALLAM,30/03/2021 à 14:01:32,30/03/2021 à 16:04:55,2 h 3 m,i.benelallam@insea.ac.ma,Organisateur
1,Chaimaa BELKARKOR,30/03/2021 à 14:01:40,30/03/2021 à 16:06:37,2 h 4 m,cbelkarkor@insea.ac.ma,Présentateur
2,Niima CHIGR,30/03/2021 à 14:01:47,30/03/2021 à 16:05:29,2 h 3 m,nchigr@insea.ac.ma,Présentateur
3,Achraf BALIJ,30/03/2021 à 14:02:06,30/03/2021 à 16:05:04,2 h 2 m,abalij@insea.ac.ma,Présentateur
4,Zakariae JAAFARI,30/03/2021 à 14:02:09,30/03/2021 à 16:05:03,2 h 2 m,zjaafari@insea.ac.ma,Présentateur
...,...,...,...,...,...,...
85,Achraf RAMOUDI,30/03/2021 à 14:38:27,30/03/2021 à 16:12:48,1 h 34 m,aramoudi@insea.ac.ma,Présentateur
86,Youness MOUZAKI,30/03/2021 à 15:17:52,30/03/2021 à 16:05:02,47 m 9 s,ymouzaki@insea.ac.ma,Présentateur
87,Hamdi HABBOUZA,30/03/2021 à 15:24:12,30/03/2021 à 16:08:04,43 m 52 s,hhabbouza@insea.ac.ma,Présentateur
88,Ayoub ELHADINE,30/03/2021 à 15:28:27,30/03/2021 à 16:01:32,33 m 4 s,aelhadine@insea.ac.ma,Présentateur


In [34]:
start_str = meta_data[3].replace("Heure de début de la réunion\t","").replace("\n","").replace(" à","")
end_str   = meta_data[4].replace("Heure de fin\t","").replace("\n","").replace(" à","")
start_course = datetime.datetime.strptime(start_str, '%d/%m/%Y %H:%M:%S')
end_course = datetime.datetime.strptime(end_str, '%d/%m/%Y %H:%M:%S')

In [35]:
mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
mycursor = mydb.cursor()
table="fact_table"
mycursor.execute(f"SELECT * FROM information_schema.columns WHERE table_schema = 'biabsence' AND table_name   = '{table}' ;")
myresult = mycursor.fetchall()
for value in myresult:
    print(value[3],":",value[7])

id_prof : integer
id_student : integer
nb_abs : integer
abs_duration : double precision
nb_lates : integer
nb_pres : integer
duration : integer
id_time : integer


In [36]:
def add_student(name,gender,email,course,major):
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"SELECT * FROM students where email ='{email}'")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO students (name,gender,email,course,major) VALUES (%s,%s,%s,%s,%s)"
        val = (name,gender,email,course,major)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"SELECT * FROM students where email ='{email}'")
        myresult = mycursor.fetchall()
        return myresult[0][0]
    
def add_professor(name,email):
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"SELECT * FROM professors where email ='{email}'")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO professors (name,email) VALUES (%s,%s)"
        val = (name,email)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"SELECT * FROM professors where email ='{email}'")
        myresult = mycursor.fetchall()
        return myresult[0][0]
    
def add_time(hour,day,month,year,date):
    mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
    mycursor = mydb.cursor()
    mycursor.execute(f"SELECT * FROM Time where year = {year} and month={month} and day={day} and hour={hour}")
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO time (hour,day,month,year,date) VALUES (%s,%s,%s,%s,%s)"
        val = (hour,day,month,year,date)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        mycursor.execute(f"SELECT * FROM time where year = {year} and month={month} and day={day} and hour={hour}")
        myresult = mycursor.fetchall()
        return myresult[0][0]

In [37]:
hour = start_course.hour
day  = start_course.day
month= start_course.month
year = start_course.year
date = start_course.strftime("%Y-%m-%d %H:%M:%S")
id_time=add_time(hour,day,month,year,date)
id_time

1

In [38]:
for index, row in df_log.iterrows():
    if row["Rôle"]=="Organisateur":
        name,email = row["Nom complet"],row["E-mail"]
        id_prof = add_professor(name,email)
        print("professor:",id_prof)

professor: 1


In [39]:
groups = df_log.groupby("E-mail")

In [49]:
mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
mycursor = mydb.cursor()
mycursor.execute(f"SELECT * FROM students")
myresult = mycursor.fetchall()
emails=[]
for result in myresult:
    emails.append(result[3])

In [50]:
print((end_course-start_course).total_seconds() / 60.0)

176.6


In [51]:

for email in emails:
    name,gender,email,course,major = None,None,email,course,None
    id_student=add_student(name,gender,email,course,major)
    try:
        data = groups.get_group(email).reset_index()
        nb_abs = 0
        #DURATION
        duration=0
        got_first_start=False
        for index, row in data.iterrows():
            start_str = row["Heure d'arrivée"].replace(" à","")
            end_str   = row["Heure de départ"].replace(" à","")
            start = datetime.datetime.strptime(start_str, '%d/%m/%Y %H:%M:%S')
            end = datetime.datetime.strptime(end_str, '%d/%m/%Y %H:%M:%S')
            duration = duration + ((end-start).total_seconds() / 60.0)
            if not got_first_start:
                got_first_start = True
                first_start = start
        if duration > 120:
            duration=120
        #NB LATES  
        if ((first_start-start_course).total_seconds() / 60.0) > 15:
            nb_lates = 1
        else:
            nb_lates = 0
        
        nb_pres = 1
        
        #ABS DURATION
        abs_duration = 120-duration
        
        #DURATION IN INTEGER IN DB!
        duration = int(duration)
        
        #is student
        print("student:",id_student)
        if duration <= 15:
            nb_abs = 1
            nb_lates=0
            nb_pres=0
    
    except KeyError as e:
        print("not found",e)
        nb_abs = 1
        abs_duration = 120
        nb_lates=0
        nb_pres=0
        duration=0
    #insert to data mart now
    try:
        mydb = psycopg2.connect(database=config["database"], user=config["user"], password=config["password"], host=config["host"], port=config["port"], options=f'-c search_path=biabsence')
        mycursor = mydb.cursor()
        sql = "INSERT INTO fact_table (id_time,id_prof,id_student,nb_abs,abs_duration,nb_lates,nb_pres,duration) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
        val = (id_time,id_prof,id_student,nb_abs,abs_duration,nb_lates,nb_pres,duration)
        mycursor.execute(sql, val)
        mydb.commit()
    except:
        continue

student: 1
student: 2
student: 3
student: 4
student: 5
student: 6
student: 7
student: 8
student: 9
student: 10
student: 11
student: 12
student: 13
student: 14
student: 15
student: 16
student: 17
student: 18
not found 'selkhyati@insea.ac.ma'
student: 20
student: 21
student: 22
student: 23
student: 24
student: 25
student: 26
student: 27
student: 28
student: 29
student: 30
student: 31
student: 32
student: 33
student: 34
student: 35
student: 36
student: 37
student: 38
student: 39
student: 40
student: 41
student: 42
student: 43
student: 44
student: 45
student: 46
student: 47
student: 48
student: 49
student: 50
student: 51
not found 'acheddad@insea.ac.ma'
student: 53
student: 54
student: 55
not found 'ahamdi@insea.ac.ma'
student: 57
student: 58
student: 59
student: 60
student: 61
student: 62
student: 63
student: 64
student: 65
student: 66
not found 'kbensetti@insea.ac.ma'
student: 68
student: 69
student: 70
not found 'msidibé@insea.ac.ma'
student: 72
student: 73
student: 74
student: 75
stude

In [53]:
os.system(f'mv "{input_file}" "/home/anas/notebook/UseCase/processed/"')

0